[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dbamman/anlp25/blob/main/10.llms/LLM_Personas.ipynb)

In this notebook, we'll use LLMs to elicit personas, first replicating Horton 2023, "[Large Language Models as Simulated Economic Agents: What Can We Learn from Homo Silicus?](https://arxiv.org/abs/2301.07543)" and then eliciting your own personas to answer questions.

Work in pairs; have one person download the base model and the other download the instruction-tuned model.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# Run this cell ASAP to download the models

# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-4B-Base", device_map="cuda", dtype="auto")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-4B", device_map="cuda", dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B")

In [ ]:
def prompt_model(messages, model, thinking=False):
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=thinking # Switches between thinking and non-thinking modes. Default is True.
    )
    
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    # conduct text completion
    generated = model.generate(
        **model_inputs,
        max_new_tokens=500
    )

    # let's break this down:
    #                      | we take the element of the batch (our batch size is 1)
    #                      |  |-----------------------------| skip our original input
    output_ids = generated[0][len(model_inputs.input_ids[0]):].tolist()

    # decode into token space
    return tokenizer.decode(output_ids, skip_special_tokens=True).strip("\n")


In [ ]:
def elicit_persona(persona_prompt, query, base=True):
    messages = [
        {"role": "system", "content": persona_prompt},
        {"role": "user", "content": query}
    ]
    print(prompt_model(messages, model))

## Prompting base model

First, let's examine's Horton's use of LLMs to elicit preferences with respect to pricing increases (originally from [Kahneman et al. 1986](https://www.jstor.org/stable/1806070)).

Run these prompts through the base model.

In [ ]:
persona_prompt = "Adopt the persona of a libertarian and answer the following query under that persona."
query = """
A hardware store has been selling snow shovels for $15. The morning after a large snowstorm, the store raises the price to $20.
Please rate this action as: 1) Completely Fair 2) Acceptable 3) Unfair 4) Very Unfair"""

elicit_persona(persona_prompt, query)

In [ ]:
persona_prompt = "Adopt the persona of a socialist and answer the following query under that persona."
query = """
A hardware store has been selling snow shovels for $15. The morning after a large snowstorm, the store raises the price to $20.
Please rate this action as: 1) Completely Fair 2) Acceptable 3) Unfair 4) Very Unfair"""

elicit_persona(persona_prompt, query)

## Prompting post-trained model

The base model has not been further post-trained with instruction or preference tuning. If you're interested in the specifics of the pre-training and post-training, you can read the [Qwen 3 technical report](https://arxiv.org/pdf/2505.09388#page=3.77).

Now try running the same inputs through an instruction-tuned model. How do the outputs change with the post-trained versions of the models?

## Constructing your own persona

Write a persona prompt to simulate yourself. Consider what details you might want to include. Experiment with the base and preference-tuned models.

Note down your own answers to these questions, then see how the model outputs compare:

1. Do you think money brings happiness?
2. Do you prefer hiking or video games?
3. Are you a morning person or a night owl?
4. What is your favorite sport to watch?
5. Do you think greater respect for authority is a good thing, a bad thing, or you don't mind?
6. How interested in politics are you? Are you 1) very interested, 2) somewhat interested, 3) not very interested, 4) not at all interested?

In [ ]:
persona_prompt = """FILL IN YOUR PERSONA HERE
"""

In [ ]:
questions = [
    "Do you think money brings happiness?",
    "Do you prefer hiking or video games?",
    "Are you a morning person or a night owl?",
    "What is your favorite sport to watch?",
    "Do you think greater respect for authority is a good thing, a bad thing, or you don't mind?",
    "How interested in politics are you? Are you 1) very interested, 2) somewhat interested, 3) not very interested, 4) not at all interested?",
]

for question in questions:
    print("=======")
    print(question)
    print()
    elicit_persona(persona_prompt, question, base=False)